## xcube Data Store Framework - Climate Data Store

*Please checkout the general introduction to xcube data stores in the Jupyter Notebook [Getting Started](./1_getting_started.ipynb) before jumping into this notebook :)* 

This notebook walks provides a walk-through demonstrating how to use xcube and the xcube plugin for the [Climate Data Store](https://cds.climate.copernicus.eu) (CDS) to read and explore temperature data from the CDS.

In order to run this notebook you need to install the `xcube_cds` plugin. You may do so by executing the following line in your terminal: 

```
$  conda install -c conda-forge xcube-cds
```

Or you can install `xcube_cds` from sources by following the instructions on https://github.com/dcs4cop/xcube-cds.

**Please note:** 
    
To access data from the Climate Data Store, you need a CDS API key. You can obtain the UID and API key as follows:

1. Create a user account on the [CDS Website](https://cds.climate.copernicus.eu/user/register).
2. Log in to the website with your user name and password.
3. Navigate to your user profile on the website. Your API key is shown at the bottom of the page.

Then export the `CDSAPI_URL` and `CDSAPI_KEY` environment variables:

```
$ export CDSAPI_URL=https://cds.climate.copernicus.eu/api/v2
$ export CDSAPI_KEY=<UID>:<API-KEY>
```

Or do it for this Notebook: (Note, we don not recommend this since this introduces a security risk!)

In [1]:
# import os
# os.environ["CDSAPI_URL"] =  https://cds.climate.copernicus.eu/api/v2
# os.environ["CDSAPI_KEY"] = <UID>:<API-KEY>

In [2]:
# mandatory imports
from xcube.core.store import find_data_store_extensions
from xcube.core.store import get_data_store_params_schema
from xcube.core.store import new_data_store

# Utilities for notebook visualization
from IPython.display import JSON
import matplotlib as mpl
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

Configure matplotlib to display graphs inline directly in the notebook and set a sensible default figure size.

In [3]:
%matplotlib inline
plt.rcParams["figure.figsize"] = 16,12

Check whether the `cds` store is among the available stores, if not please follow the installation information from the top of this notebook. 

In [4]:
JSON({e.name: e.metadata for e in find_data_store_extensions()})

<IPython.core.display.JSON object>

Usually we need more information to get the actual data store object. Which data store parameters are available for `cds`?

In [5]:
get_data_store_params_schema('cds')

Provide mandatory parameters to instantiate the store class:

In [6]:
store = new_data_store('cds')
store

Which datasets are provided? (the list may contain both gridded and vector datasets):

In [7]:
JSON(list(store.get_data_ids()))

<IPython.core.display.JSON object>

Get more info about a specific dataset. This includes a description of the possible open formats:

In [8]:
store.describe_data('reanalysis-era5-single-levels-monthly-means:monthly_averaged_reanalysis')

There are 4 required parameters, so we need to provide them to open a dataset:

In [9]:
dataset = store.open_data('reanalysis-era5-single-levels-monthly-means:monthly_averaged_reanalysis', 
                          variable_names=['2m_temperature'], 
                          bbox=[-10, 45, 40, 65], 
                          spatial_res=0.25, 
                          time_range=['2001-01-01', '2010-12-31'])
dataset

xcube-cds version 0.9.0.dev0


Exception: Missing/incomplete configuration file: C:\Users\Norman/.cdsapirc

Plot the differences between successive time points in the dataset. We can see that the times are monotonically increasing (all the difference values are positive), but not equally spaced, since months are not all of the same length. The lowest values correspond to February; the four-year leap year cycle can also be discerned.

In [ ]:
dataset.time.diff(dim='time').plot.line(figsize=(20, 4))

We can explore these data by plotting a temperature map for selected time points. First, we select January 2001. Land areas – and mountain ranges in particular – show up on the map as colder regions.

In [ ]:
t2m_2001_jan = dataset.t2m.sel(time='2001-01-01 00:00:00', method='nearest')
t2m_2001_jan.plot.imshow(vmin=260, vmax=300, figsize=(14, 8))

For a more elegant and informative map, we define a function to plot a customized orthographic projection with overlaid coastlines and a grid.


In [ ]:
def plot_map(data_array, colour_scale=(None, None), cmap=None):
    mid_lat = 55
    mid_lon = 15
    proj = ccrs.Orthographic(central_longitude=mid_lon, central_latitude=mid_lat)
    ax = plt.axes(projection=proj)
    im = data_array.plot.imshow(ax=ax, transform=ccrs.PlateCarree(), add_colorbar=False, vmin=colour_scale[0], vmax=colour_scale[1], cmap=cmap)
    ax.coastlines(resolution='50m')
    ax.set_extent((-1.9e6, 1.9e6, -1.1e6, 1.3e6), crs=proj)
    draw_labels = False  # only supported from cartopy 0.18.0 onwards
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=draw_labels, color='#000000', alpha=0.5, linestyle='--')
    gl.xlocator = mpl.ticker.FixedLocator([-10, 0, 10, 20, 30, 40])
    gl.ylocator = mpl.ticker.FixedLocator([40, 45, 50, 55, 60, 65])
    plt.colorbar(im, fraction=0.027, pad=0.04)

common_scale = (260, 300)

In [ ]:
plot_map(t2m_2001_jan, common_scale)
